<a href="https://colab.research.google.com/github/dvignoles/nyc-transpo/blob/master/nyc_transpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import requests, zipfile, io, os, re, csv

TAXI_CSV_PATH = "./taxi_data"
BIKE_CSV_PATH = "./bike_data"

# We will use with 2013-2018 June data since June 2013 is when Citi Bike was started, and 2013-2018 data is available for both taxi and bikes.
# We will also filter down to the first week of the month, because the data is an absolute unit
YEARS = ["2013", "2014", "2015", "2016", "2017", "2018"]

In [2]:
# Set up directories and util functions to store the downloaded csv files

if os.path.isdir(TAXI_CSV_PATH) is False:
  os.mkdir(TAXI_CSV_PATH)
if os.path.isdir(BIKE_CSV_PATH) is False:
  os.mkdir(BIKE_CSV_PATH)

  
def save_csv(file_path, http_data):
    with open(file_path, "w") as f:
      writer = csv.writer(f)
      reader = csv.reader(http_data.text.splitlines())
      for row in reader:
        writer.writerow(row)
        
def get_csv_name(data_type, year, two_weeks_only=False):
  if data_type is "taxi":
    return TAXI_CSV_PATH + "/" + year + "-06" + ("_parsed" if two_weeks_only is True else "") +  ".csv"
  elif data_type is "bike":
    return BIKE_CSV_PATH + "/" + year + "-06" + ("_parsed" if two_weeks_only is True else "") + ".csv"
  else:
    raise Exception("Data type must be taxi or bike")

# Taxi Dataframes
Since these downloads take a long time, do NOT reset all run times when it asks you to! It will wipe all downloaded files, and you'll have to download them again!

In [3]:
# Download and save the files first so they don't have to be redownloaded every time

for n in YEARS:
  file_path = get_csv_name("taxi", n, False)
  if os.path.isfile(file_path) is False:
    data = requests.get("https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_" + n + "-06.csv")
    save_csv(file_path, data)

# Load the actual data

dtypes = {
    "vendor_id": "str",
    "VendorID": "str",
    "pickup_datetime": "str",
    "dropoff_datetime": "str",
    "tpep_pickup_datetime": "str",
    "tpep_dropoff_datetime": "str",
    "passenger_count": np.int32,
    "trip_distance": np.float32,
    "pickup_longitude": np.float32,
    "pickup_latitude": np.float32,
    "PULocationID": np.int32,
    "DOLocationID": np.int32,
    "rate_code": np.int32,
    "RateCodeID": np.int32,
    "store_and_fwd_flag": "str",
    "dropoff_longitude": np.float32,
    "dropoff_latitude": np.float32,
    "payment_type": "str",
    "fare_amount": np.float32,
    "surcharge": np.float32,
    "mta_tax": np.float32,
    "tip_amount": np.float32,
    "tolls_amount": np.float32,
    "total_amount": np.float32
}

def get_parsed_taxi_df(year):
  # See if the sorted data was saved before, if so use it
  sorted_file_path = get_csv_name("taxi", year, True)
  if os.path.isfile(sorted_file_path) is True:
    df_toreturn = pd.read_csv(sorted_file_path, dtype=dtypes)
    df_toreturn['pickup_datetime_parsed'] = pd.to_datetime(df_toreturn['pickup_datetime_parsed'])
    return df_toreturn
  
  # Otherwise, load the full file, sort it, and save it for future reference
  print("getting full data")
  df_temp = pd.read_csv(get_csv_name("taxi", year, False), dtype=dtypes)
  
  # Sort by date
  df_temp.sort_values(by=df_temp.columns[1], inplace=True, kind='mergesort')
  df_temp['pickup_datetime_parsed'] = pd.to_datetime(df_temp.iloc[:,1])
  
  # Get only the first week of the month
  df_temp = df_temp[(df_temp['pickup_datetime_parsed'] > (year + '-06-01')) & (df_temp['pickup_datetime_parsed'] < (year + '-06-08'))]
  df_temp.to_csv(sorted_file_path)
  return df_temp
  
# The util function MUST be used to save memory while loading in the entire dataframes for each year.
# IGNORE MEMORY LIMIT WARNINGS! The data is garbage collected every time the function exits for each year.

df_taxi_2013 = get_parsed_taxi_df("2013")
df_taxi_2014 = get_parsed_taxi_df("2014")
df_taxi_2015 = get_parsed_taxi_df("2015")
df_taxi_2016 = get_parsed_taxi_df("2016")
df_taxi_2017 = get_parsed_taxi_df("2017")
df_taxi_2018 = get_parsed_taxi_df("2018")

getting full data
getting full data
getting full data
getting full data
getting full data
getting full data


In [10]:
df_taxi_2018.tail()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_datetime_parsed
2103666,2,2018-06-07 23:59:58,2018-06-08 00:07:43,1,1.19,1,N,164,43,2,7.0,0.5,0.5,0.00,0.0,0.3,8.300000,2018-06-07 23:59:58
2094722,2,2018-06-07 23:59:59,2018-06-08 00:19:05,2,4.78,1,N,231,17,1,17.5,0.5,0.5,3.76,0.0,0.3,22.559999,2018-06-07 23:59:59
2103590,1,2018-06-07 23:59:59,2018-06-08 00:01:56,1,0.70,1,N,141,236,1,4.0,0.5,0.5,1.05,0.0,0.3,6.350000,2018-06-07 23:59:59
2113411,2,2018-06-07 23:59:59,2018-06-08 00:10:43,5,5.68,1,N,13,137,1,17.5,0.5,0.5,3.76,0.0,0.3,22.559999,2018-06-07 23:59:59
2116846,2,2018-06-07 23:59:59,2018-06-08 00:14:38,1,2.53,1,N,231,79,1,12.0,0.5,0.5,2.00,0.0,0.3,15.300000,2018-06-07 23:59:59


# Citi Bike Dataframes
Since Citi Bike does not directly provide csv files, we must handle the zip files, extract, and load them accordingly.

In [5]:
reg_pattern = re.compile("^\d+.*\.csv$")

for n in YEARS:
  # See if the file exists
  file_path = get_csv_name("bike", n)
  if os.path.isfile(file_path) is True:
    continue

  # If not, request and save it
  req_url = ""
  if n != "2017" and n != "2018":
    req_url = "https://s3.amazonaws.com/tripdata/" + n + "06-citibike-tripdata.zip"
  else:
    req_url = "https://s3.amazonaws.com/tripdata/" + n + "06-citibike-tripdata.csv.zip"
    
  # Handle the ZIP
  z = zipfile.ZipFile(io.BytesIO(requests.get(req_url).content))
  orig_file_name = list(filter(reg_pattern.match, z.namelist()))[0]
  z.extract(orig_file_name)
  os.rename(orig_file_name, get_csv_name("bike", n)) # Renaming the file will automatically remove the original, extracted file

In [6]:
# The dtypes can't be passed in atm because some columns that should be integers have non-number strings in them
# Some columns that should be integers are floats instead because pandas can't put NaN values in integer columns
"""
dtypes = {
    "tripduration": np.int32,
    "starttime": "str",
    "stoptime": "str",
    "start station id": np.float32,
    "start station name": "str",
    "start station latitude": np.float32,
    "start station longitude": np.float32,
    "end station id": np.float32,
    "end station name": "str",
    "end station latitude": np.float32,
    "end station longitude": np.float32,
    "bikeid": np.int32,
    "usertype": "str",
    "birth year": np.float32,
    "gender": np.int32
}
"""


df_bike_2013_orig = pd.read_csv(get_csv_name("bike", "2013"))
df_bike_2014_orig = pd.read_csv(get_csv_name("bike", "2014"))
df_bike_2015_orig = pd.read_csv(get_csv_name("bike", "2015"))
df_bike_2016_orig = pd.read_csv(get_csv_name("bike", "2016"))
df_bike_2017_orig = pd.read_csv(get_csv_name("bike", "2017"))
df_bike_2018_orig = pd.read_csv(get_csv_name("bike", "2018"))

In [7]:
# Convert to datetime. This was really annoying with the different formats - if the format is unspecified, the functions never stop running.
# Also must assert that the last row of data has a start time greater than the second week, to make sure we have the full first 2 weeks of data

# 2013
df_bike_2013_orig['starttime_parsed'] = pd.to_datetime(df_bike_2013_orig['starttime'])
df_bike_2013_orig['stoptime_parsed'] = pd.to_datetime(df_bike_2013_orig['stoptime'])
assert df_bike_2013_orig.iloc[-1].starttime_parsed > datetime.strptime('2013-06-07', '%Y-%m-%d'), "Missing full 1 week of data for 2013"

# 2014
df_bike_2014_orig['starttime_parsed'] = pd.to_datetime(df_bike_2014_orig['starttime'])
df_bike_2014_orig['stoptime_parsed'] = pd.to_datetime(df_bike_2014_orig['stoptime'])
assert df_bike_2014_orig.iloc[-1].starttime_parsed > datetime.strptime('2014-06-07', '%Y-%m-%d'), "Missing full 1 week of data for 2014"

# 2015
df_bike_2015_orig['starttime_parsed'] = pd.to_datetime(df_bike_2015_orig["starttime"], format="%m/%d/%Y %H:%M")
df_bike_2015_orig['stoptime_parsed'] = pd.to_datetime(df_bike_2015_orig['stoptime'], format="%m/%d/%Y %H:%M")
assert df_bike_2015_orig.iloc[-1].starttime_parsed > datetime.strptime('2015-06-07', '%Y-%m-%d'), "Missing full 1 week of data for 2015"

# 2016
df_bike_2016_orig['starttime_parsed'] = pd.to_datetime(df_bike_2016_orig['starttime'], format="%m/%d/%Y %H:%M:%S")
df_bike_2016_orig['stoptime_parsed'] = pd.to_datetime(df_bike_2016_orig['stoptime'], format="%m/%d/%Y %H:%M:%S")
assert df_bike_2016_orig.iloc[-1].starttime_parsed > datetime.strptime('2016-06-07', '%Y-%m-%d'), "Missing full 1 week of data for 2016"

# 2017
df_bike_2017_orig['starttime_parsed'] = pd.to_datetime(df_bike_2017_orig['starttime'], format="%Y-%m-%d %H:%M:%S")
df_bike_2017_orig['stoptime_parsed'] = pd.to_datetime(df_bike_2017_orig['stoptime'], format="%Y-%m-%d %H:%M:%S")
assert df_bike_2017_orig.iloc[-1].starttime_parsed > datetime.strptime('2017-06-07', '%Y-%m-%d'), "Missing full 1 week of data for 2017"

# 2018
df_bike_2018_orig['starttime_parsed'] = pd.to_datetime(df_bike_2018_orig['starttime'], format="%Y-%m-%d %H:%M:%S")
df_bike_2018_orig['stoptime_parsed'] = pd.to_datetime(df_bike_2018_orig['stoptime'], format="%Y-%m-%d %H:%M:%S")
assert df_bike_2018_orig.iloc[-1].starttime_parsed > datetime.strptime('2018-06-07', '%Y-%m-%d'), "Missing full 1 week of data for 2018"

In [8]:
# Our main Citi Bike dataframes we'll be using, with a full week of data for each year

df_bike_2013 = df_bike_2013_orig[(df_bike_2013_orig['starttime_parsed'] > '2013-06-01') & (df_bike_2013_orig['starttime_parsed'] < '2013-06-08')]
df_bike_2014 = df_bike_2014_orig[(df_bike_2014_orig['starttime_parsed'] > '2014-06-01') & (df_bike_2014_orig['starttime_parsed'] < '2014-06-08')]
df_bike_2015 = df_bike_2015_orig[(df_bike_2015_orig['starttime_parsed'] > '2015-06-01') & (df_bike_2015_orig['starttime_parsed'] < '2015-06-08')]
df_bike_2016 = df_bike_2016_orig[(df_bike_2016_orig['starttime_parsed'] > '2016-06-01') & (df_bike_2016_orig['starttime_parsed'] < '2016-06-08')]
df_bike_2017 = df_bike_2017_orig[(df_bike_2017_orig['starttime_parsed'] > '2017-06-01') & (df_bike_2017_orig['starttime_parsed'] < '2017-06-08')]
df_bike_2018 = df_bike_2018_orig[(df_bike_2018_orig['starttime_parsed'] > '2018-06-01') & (df_bike_2018_orig['starttime_parsed'] < '2018-06-08')]


In [33]:
df_bike_2014.head(50)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,starttime_parsed,stoptime_parsed
0,520,2014-06-01 00:00:02,2014-06-01 00:08:42,358,Christopher St & Greenwich St,40.732916,-74.007114,426,West St & Chambers St,40.717548,-74.013221,18840,Subscriber,1979,1,2014-06-01 00:00:02,2014-06-01 00:08:42
1,520,2014-06-01 00:00:27,2014-06-01 00:09:07,335,Washington Pl & Broadway,40.729039,-73.994046,265,Stanton St & Chrystie St,40.722293,-73.991475,17442,Customer,\N,0,2014-06-01 00:00:27,2014-06-01 00:09:07
2,414,2014-06-01 00:00:32,2014-06-01 00:07:26,439,E 4 St & 2 Ave,40.726281,-73.989780,368,Carmine St & 6 Ave,40.730386,-74.002150,16447,Subscriber,1980,1,2014-06-01 00:00:32,2014-06-01 00:07:26
3,310,2014-06-01 00:00:34,2014-06-01 00:05:44,463,9 Ave & W 16 St,40.742065,-74.004432,380,W 4 St & 7 Ave S,40.734011,-74.002939,18218,Subscriber,1984,1,2014-06-01 00:00:34,2014-06-01 00:05:44
4,457,2014-06-01 00:00:35,2014-06-01 00:08:12,352,W 56 St & 6 Ave,40.763406,-73.977225,305,E 58 St & 3 Ave,40.760958,-73.967245,18115,Subscriber,1969,1,2014-06-01 00:00:35,2014-06-01 00:08:12
5,399,2014-06-01 00:00:43,2014-06-01 00:07:22,293,Lafayette St & E 8 St,40.730287,-73.990765,247,Perry St & Bleecker St,40.735354,-74.004831,18527,Subscriber,1970,1,2014-06-01 00:00:43,2014-06-01 00:07:22
6,1280,2014-06-01 00:00:45,2014-06-01 00:22:05,168,W 18 St & 6 Ave,40.739713,-73.994564,341,Stanton St & Mangin St,40.717821,-73.976289,18766,Subscriber,1989,1,2014-06-01 00:00:45,2014-06-01 00:22:05
7,973,2014-06-01 00:00:52,2014-06-01 00:17:05,453,W 22 St & 8 Ave,40.744751,-73.999154,447,8 Ave & W 52 St,40.763707,-73.985162,21391,Customer,\N,0,2014-06-01 00:00:52,2014-06-01 00:17:05
8,500,2014-06-01 00:00:58,2014-06-01 00:09:18,498,Broadway & W 32 St,40.748549,-73.988084,488,W 39 St & 9 Ave,40.756458,-73.993722,18592,Subscriber,1980,1,2014-06-01 00:00:58,2014-06-01 00:09:18
9,253,2014-06-01 00:00:58,2014-06-01 00:05:11,312,Allen St & E Houston St,40.722055,-73.989111,473,Rivington St & Chrystie St,40.721101,-73.991925,19377,Subscriber,1983,1,2014-06-01 00:00:58,2014-06-01 00:05:11


In [27]:
citi_avg_trip_durations = []

citi_avg_trip_durations.append(df_bike_2013.tripduration.mean())
citi_avg_trip_durations.append(df_bike_2014.tripduration.mean())
citi_avg_trip_durations.append(df_bike_2015.tripduration.mean())
citi_avg_trip_durations.append(df_bike_2016.tripduration.mean())
citi_avg_trip_durations.append(df_bike_2017.tripduration.mean())
citi_avg_trip_durations.append(df_bike_2018.tripduration.mean())

print(citi_avg_trip_durations)

[2334.452260073118, 907.9234496339398, 889.1972157265556, 1023.7191479376997, 973.301784785734, 1161.3499848084173]


In [37]:
# had to avoid 2014 here, as there are some issues with the birthdate field in that dataframe. Will clean.
# yo people in their 40s love citibikes

citi_avg_age = []

citi_avg_age.append(df_bike_2013["birth year"].mean())
citi_avg_age.append("NaN")
citi_avg_age.append(df_bike_2015["birth year"].mean())
citi_avg_age.append(df_bike_2016["birth year"].mean())
citi_avg_age.append(df_bike_2017["birth year"].mean())
citi_avg_age.append(df_bike_2018["birth year"].mean())

print(citi_avg_age)

[1973.7418080639952, 'NaN', 1976.2964147385587, 1977.4734717561873, 1978.7014382409977, 1978.7112451471332]
